In [1]:
import matplotlib.pyplot as plt
from tqdm import trange
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np


# Assuming the npy files are named 'delta_0.npy', 'delta_1.npy', etc.

start_n = 0
end_n = 799

# Specify the index of the item to plot
item_index = 2500

# Initialize a list to store the item values
item_values = []
vector_values = []

# Load the item values from the npy files
for i in trange(start_n, end_n):
    vector = np.load(f'/user/as6154/dissert/L12_half_data/delta_{i}.npy')

    item_values.append(vector[item_index])
    vector_values.append(vector)

vector_values = np.array(vector_values)
# # Plot the item values
# plt.figure(figsize=(10, 6))
# plt.plot(item_values, label=f'Item {item_index}')

# plt.xlabel('File Index')
# plt.ylabel('Item Value')
# plt.title(f'Item {item_index} from NPY Files')
# plt.legend()
# plt.show()

100%|██████████| 799/799 [00:00<00:00, 1685.71it/s]


In [2]:
len(vector)

4096

In [3]:
import pickle
number_of_bins = 100000
dir_path = "/user/as6154/dissert/szhalf_L12_delta_-2_to_0_interval_400_secondDelta_1_to_1_interval_1_data"

# Define the path to save the pickle file
pickle_file_path = f'{dir_path}/{number_of_bins}_binned_values.pkl'
with open(pickle_file_path, 'rb') as f:
    binned_values_loaded = pickle.load(f)

print(f"Binned values loaded from pickle file (first 10): {binned_values_loaded[:10]}")
print(f"Binned values (last 10): {binned_values_loaded[-10:]}")  # Display first 10 binned values for reference

Binned values loaded from pickle file (first 10): [[99999 11065 11065 ... 11065 11065 11065]
 [99999 11065 11065 ... 11065 11065 11065]
 [99999 11065 11065 ... 11065 11065 11065]
 ...
 [99999 11065 11065 ... 11065 11065 11065]
 [99999 11065 11065 ... 11065 11065 11065]
 [99999 11065 11065 ... 11065 11065 11065]]
Binned values (last 10): [[11065 11065 11065 ... 11065 11065 11065]
 [11065 11065 11065 ... 11065 11065 11065]
 [11065 11065 11065 ... 11065 11065 11065]
 ...
 [11065 11065 11065 ... 11065 11065 11065]
 [11065 11065 11065 ... 11065 11065 11065]
 [11065 11065 11065 ... 11065 11065 11065]]


In [4]:

from sklearn.model_selection import train_test_split

sequence_length = 10
batch_size = 4

class HighDimensionalDataset(Dataset):
    def __init__(self, data, n_items = 1000, random = False):
        self.data = data
        self.random = random
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.random:
            start_index = np.random.randint(0, len(self.data) - sequence_length)
        else:
            start_index = min(idx, len(self.data) - sequence_length)
        sequence = self.data[start_index : start_index + sequence_length]
        return torch.tensor(sequence, dtype=torch.float32)



# Split the data into training and testing sets
train_data = binned_values_loaded
test_data = binned_values_loaded

# TODO: But this test_dataset would also work if it's overfitting?
train_dataset = HighDimensionalDataset(train_data, n_items = 1000, random = True)
test_dataset = HighDimensionalDataset(test_data, n_items = 1000, random = False)


# Create dataloaders for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [5]:
train_dataset[100].shape

torch.Size([10, 4096])

Setting:

1. We have a sequence 800 data, that is across delta changing, each item is vector dim 4096.
2. I want to predict next data, conditioning on previous data. 
3. So my input and output is dim 4096. 

Idea is to 


-----

Normal language models take in batch of text, text is in high dimension, but it's a number alone.
so it's BxN.

Our's is BxNxD.

So i need to embed the D to 1D?


Can we embed continous instead of quantize? 
- but we need to tokenize first right?
- unless i train my own tokenizer with bpe?
- so we shouldnt?

In [6]:
train_dataset[0].shape[1]

4096

In [7]:
# Garg inspired model
from transformers import GPT2Config, GPT2Model
import torch.nn as nn


class Transformer(nn.Module):
    def __init__(self, n_dims, n_positions, n_embd=4096, n_layer=12, n_head=4):
        super(Transformer, self).__init__()
        configuration = GPT2Config(
            n_positions=2 * n_positions,
            n_embd=n_embd,
            n_layer=n_layer,
            n_head=n_head,
            resid_pdrop=0.0,
            embd_pdrop=0.0,
            attn_pdrop=0.0,
            use_cache=False,
        )
        self.name = f"gpt2_embd={n_embd}_layer={n_layer}_head={n_head}"

        self.n_positions = n_positions
        self.n_dims = n_dims
        
        self._read_in = nn.Linear(n_dims, n_embd)
        self._backbone = GPT2Model(configuration)
        self._read_out = nn.Linear(n_embd, n_dims)

    def forward(self, x):
        embeds = self._read_in(x)
        output = self._backbone(inputs_embeds = embeds).last_hidden_state
        prediction = self._read_out(output)
        return prediction
    
n_dims = train_dataset[0].shape[1]
n_positions = 1024

model = Transformer(n_dims, n_positions)

input_data = torch.randn(batch_size, sequence_length, n_dims)

output = model(input_data)
print(output.shape)

[2024-06-26 22:06:00,446] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
torch.Size([4, 10, 4096])


In [8]:
next(iter(train_dataloader)).shape

torch.Size([4, 10, 4096])

In [9]:
next(iter(test_dataloader)).shape

torch.Size([1, 10, 4096])

In [10]:
import wandb
from transformers import AdamW, get_linear_schedule_with_warmup
import pandas as pd


wandb.init(project="GPT_QFT")
loss_fn = nn.MSELoss()
device = "cuda:1"
num_epochs = 100
eval_steps = 100

optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4)

# optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(train_dataloader) * num_epochs)



def evaluate_model(model, test_dataloader, device, loss_fn):
    loss_df = pd.DataFrame()
    model.eval()
    with torch.no_grad():
        loss_array = []
        for i, (test_input_seq) in enumerate(test_dataloader):
            test_target_seq = test_input_seq[:, 1:, :]
            test_input_seq = test_input_seq.to(device)
            test_target_seq = test_target_seq.to(device)

            test_output = model(test_input_seq)
            test_output = test_output[:, 1:, :]

            test_loss = loss_fn(test_output, test_target_seq)
            loss_array.append((i, test_loss.item()))
        loss_df = pd.DataFrame(loss_array, columns=['Index', 'Loss'])
        loss_df.drop(columns=['Index'], inplace=True)
        loss_df = loss_df.transpose()
        loss_df.to_csv('loss_data_n2_100000.csv', mode='a', header=False, index=False)

        loss_array.sort(key=lambda x: x[1], reverse=True)
        print(f'Highest Loss at index: {loss_array[0][0]}, Loss: {loss_array[0][1]}')
        print(f'Lowest Loss at index: {loss_array[-1][0]}, Loss: {loss_array[-1][1]}')
        wandb.log({"Highest Loss Index": loss_array[0][0], "Lowest Loss Index": loss_array[-1][0]})
    model.train()

for epoch in trange(num_epochs):
    for i, (input_seq) in enumerate(train_dataloader):

        target_seq = input_seq[:, 1:, :]

        model.to(device)
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)
        output = model(input_seq)
        output = output[:, 1:, :]

        loss = loss_fn(output, target_seq)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        if (i+1) % eval_steps == 0:
            evaluate_model(model, test_dataloader, device, loss_fn)
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {loss.item()}')
            wandb.log({"Loss": loss.item()})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andrewsiah (hong-exploration). Use `wandb login --relogin` to force relogin


  1%|          | 1/100 [00:28<46:14, 28.02s/it]

Highest Loss at index: 0, Loss: 124377784.0
Lowest Loss at index: 266, Loss: 123781632.0
Epoch [1/100], Step [100/100], Loss: 124013880.0


  2%|▏         | 2/100 [00:53<43:18, 26.51s/it]

Highest Loss at index: 0, Loss: 123385712.0
Lowest Loss at index: 266, Loss: 122791840.0
Epoch [2/100], Step [100/100], Loss: 123163096.0


...

Highest Loss at index: 0, Loss: 41194104.0
Lowest Loss at index: 266, Loss: 40846812.0
Epoch [98/100], Step [100/100], Loss: 41041440.0


 99%|█████████▉| 99/100 [43:31<00:25, 25.54s/it]

Highest Loss at index: 0, Loss: 41168480.0
Lowest Loss at index: 266, Loss: 40821296.0
Epoch [99/100], Step [100/100], Loss: 41097188.0


100%|██████████| 100/100 [43:56<00:00, 26.37s/it]

Highest Loss at index: 0, Loss: 41159860.0
Lowest Loss at index: 266, Loss: 40812708.0
Epoch [100/100], Step [100/100], Loss: 40955736.0
